# Surrounding Landscape

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:




In [33]:
import ee
import geemap
from gee_0_utils import *

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

method = 'mapbiomas'
biomass_raw = (ee.Image(f"projects/sat-io/open-datasets/ESA/ESA_CCI_AGB/CCI_BIOMASS_100m_AGB_{last_year}_v51")
           .select("AGB").rename(f"ESA_CCI_{last_year}"))

## Surrounding mature forest biomass

- Mean biomass of surrounding mature forests (interpolated with Gaussian kernel)
- Total surrounding mature forest cover (normalized from 0-1) at a 500m distance

In [34]:
if method == 'tmf':
    transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(roi)
    mature_mask = transition.eq(10).selfMask()
    age = ee.Image(f"{data_folder}/tmf_{last_year}")

elif method == 'mapbiomas':
    lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
                .select([f"classification_{year}" for year in config.range_1985_2020])
                .byte()
                .rename([str(year) for year in config.range_1985_2020]))
    mature_mask = lulc.eq(3).reduce(ee.Reducer.allNonZero()).selfMask()
    age = ee.Image(f"{data_folder}/mapbiomas_{last_year}")

mature_biomass = biomass_raw.updateMask(mature_mask).rename("mature_biomass")

# keep only forests that are 1500m away from any non-forest area
edge_detec = mature_mask.unmask(-1).zeroCrossing()
distance_to_forest_edge = edge_detec.fastDistanceTransform(1024, 'pixels').sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).toInt16().rename("dist")

sur_cover = mature_mask.unmask(0).focalMean(radius = 500, units = "meters").float().updateMask(age).rename("sur_cover")

Export at a 100m resolution (same as ESA CCI Biomass) to argue for the distance-to-edge effect.

In [ ]:
# export_image(distance_to_forest_edge, "distance_to_forest_edge", region = roi, scale = 30)
# export_image(sur_cover, "sur_cover_500m", region = roi, scale = 30)
# export_image(mature_biomass, "mature_biomass", region = roi, scale = 100)

## Biomass of nearest mature forest

In [26]:


distance_gt_1000 = distance_to_forest_edge.gt(1000).selfMask()
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
mature_biomass_exclude_edge = mature_biomass.updateMask(distance_gt_1000).rename("mature_biomass")
mature_biomass_resampled = mature_biomass_exclude_edge.resample('bilinear').reproject(crs='EPSG:4326', scale = 1000)

edge_detec = mature_biomass_exclude_edge.unmask(-1).zeroCrossing()
distance_to_deep_forest = edge_detec.fastDistanceTransform(1024, 'pixels').sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("dist")

distance_to_deep_forest = distance_to_deep_forest.updateMask(age).add(1000)

distances_vector = distance_to_deep_forest.sample(
    region = roi,
    scale = 1000,
    projection = "EPSG:4326",
    geometries = True
)

# # Export task to Google Drive
# task = ee.batch.Export.table.toAsset(
#     collection = distances_vector,
#     description = "distances_vector",
#     assetId=f"{data_folder}/distances_vector"
# )

# task.start()


In [44]:
distances_vector = ee.FeatureCollection(f"{data_folder}/distances_vector")

def create_buffer(feature):
    distance = feature.getNumber('dist')
    buffer = feature.geometry().buffer(distance)
    return feature.setGeometry(buffer)

# Apply the function to each feature in the FeatureCollection
buffered_features = distances_vector.map(create_buffer)

nearest_mature_biomass = mature_biomass_resampled.reduceRegions(
    collection = buffered_features,
    reducer = ee.Reducer.firstNonNull(),
    crs = "EPSG:4326",
    scale = 1000
)

# Export task to Google Drive
task = ee.batch.Export.table.toAsset(
    collection = nearest_mature_biomass,
    description = "nearest_mature_biomass",
    assetId=f"{data_folder}/nearest_mature_biomass"
)

task.start()

In [39]:
nearest_mature_biomass = ee.FeatureCollection(f"{data_folder}/nearest_mature_biomass")

nearest_mature_biomass_centroids = nearest_mature_biomass.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))

Map = geemap.Map()
Map.addLayer(nearest_mature_biomass_centroids, {}, 'nearest_mature_biomass_centroids')
Map

In [ ]:
# Convert the FeatureCollection to an image
nearest_mature_biomass_image = nearest_mature_biomass_centroids.reduceToImage(["mean"], ee.Reducer.first()).byte().rename("nearest_mature_biomass")

export_image(nearest_mature_biomass_image, "nearest_mature_biomass_image", region = roi, scale = 1000)

# nearest_mature_biomass_image = ee.Image("projects/amazon-forest-regrowth/assets/nearest_mature_biomass_image1")


In [41]:

# distance_gt_1000 = distance_to_forest_edge.gt(1000).selfMask()
# mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
# mature_biomass_exclude_edge = mature_biomass.updateMask(distance_gt_1000).rename("mature_biomass")
# mature_biomass_resampled = mature_biomass_exclude_edge.resample('bilinear').reproject(crs='EPSG:4326', scale = 1000)
# # mature_biomass_resampled = mature_biomass_exclude_edge.resample('bilinear').reproject(crs='EPSG:4326', scale = 1000)


# # nearest_mature_biomass = ee.FeatureCollection(f"{data_folder}/nearest_mature_biomass")

# # nearest_mature_biomass_centroids = nearest_mature_biomass.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))
# vector = ee.FeatureCollection(f"{data_folder}/distances_vector")

# map = geemap.Map()
# map.addLayer(vector, {}, "distances_vector")
# map.addLayer(mature_biomass_exclude_edge,  {'min': 150, 'max': 260, 'palette': ['yellow', 'green', 'blue']}, "mature_biomass_exclude_edge")
# map.addLayer(mature_biomass_resampled, {'min': 150, 'max': 260, 'palette': ['yellow', 'green', 'blue']}, "mature_biomass_resampled")
# map

## Amazon Quarters
Mean Biomass Value Per Climatic Region

In [47]:
# Load the mature biomass image
mature_biomass_exclude_edge = ee.Image(f"{data_folder}/mapbiomas/mature_biomass_exclude_edge_1500m")

# Load the individual feature collections and combine them into a single FeatureCollection
quarters = ee.FeatureCollection([
    ee.FeatureCollection(f"{data_folder}/raw/NE_outline_no_overlap").map(lambda f: f.set('sector', 1)),
    ee.FeatureCollection(f"{data_folder}/raw/NW_outline_no_overlap").map(lambda f: f.set('sector', 2)),
    ee.FeatureCollection(f"{data_folder}/raw/SW_outline_no_overlap").map(lambda f: f.set('sector', 3)),
    ee.FeatureCollection(f"{data_folder}/raw/SE_outline_no_overlap").map(lambda f: f.set('sector', 4)),
]).flatten()

ecoregions = (
    ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
    .filterBounds(roi)
    .map(lambda feature: feature.intersection(roi))
)

# Function to get mean biomass per Feature
def create_biomass_image(feature):
    mean_biomass = mature_biomass.reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = feature.geometry(),
        scale = 500,
        maxPixels = 1e12
    )
    return feature.set(mean_biomass)

# Apply the function to each feature in the FeatureCollection
mean_biomass_quarters = quarters.map(create_biomass_image)
mean_biomass_ecoregions = ecoregions.map(create_biomass_image)

# Helper function to create an image from a property in the feature collection
def create_property_image(feature_collection, property_name):
    return feature_collection.reduceToImage(
        properties = [property_name],
        reducer = ee.Reducer.first()
    ).rename(property_name)

# Create images for the 'mature_biomass' and 'sector' properties
quarter_biomass = create_property_image(mean_biomass_quarters, 'mature_biomass').toInt16()
quarter = create_property_image(mean_biomass_quarters, 'sector').byte()
ecoreg_biomass = create_property_image(mean_biomass_ecoregions, 'mature_biomass').toInt16()
ecoreg = create_property_image(mean_biomass_ecoregions, 'ECO_ID').byte()

quarters_ecoreg_biomass = ee.Image.cat([quarter_biomass, quarter, ecoreg_biomass, ecoreg]).rename(["quarter_biomass", "quarter", "ecoreg_biomass", "ecoreg"])

In [48]:
export_image(quarters_ecoreg_biomass, "quarters_ecoreg_biomass", region = roi, scale = 5000)